In [1]:
from os import listdir
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from tqdm.notebook import tqdm

In [2]:
# Парсинг условий продажи

# url = 'https://www.tomsk.ru09.ru/realty?subaction=detail&id=4414402'
# test_soup = get_soup_by_url(url)

# for i in df.index[0:10]:
#     test_soup = get_soup_by_url(i)
#     if
#     print(test_soup.find('sup').parent.text)

# get_soup_by_url(df.index[0])
# requests.get(df.index[0])

In [3]:
def get_soup_by_url(url):
    
    html_ = requests.get(url).text
    soup = BeautifulSoup(html_, 'lxml')
    
    return soup    

In [4]:
# Получаем номер последней страницы
def get_number_last_page():
    
    soup = get_soup_by_url('https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page=1')
    number_last_page = int(soup.find('td', {'class':'pager_pages'}).find_all('a')[4].text)
    
    return number_last_page

In [5]:
def find_district_field(keys):
    
    for i, j in enumerate(keys):
        if 'район' in j:
            break
    return i

In [6]:
def parse_apartment(url):
#     headers = {'User-Agent: Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
#     start_time = time.time()
    
    soup = get_soup_by_url(url)
    
    keys = [i.find('span').text.replace('\xa0','').lower() for i in soup.find_all('tr', {'class': 'realty_detail_attr'})]
    
    district_idx = find_district_field(keys)
    items = {'район': keys[district_idx]}
    
    keys = [j for i, j in enumerate(keys) if i not in (district_idx - 1, district_idx)]
    values = [i.text.replace('\xa0', ' ') for i in soup.find_all(class_='nowrap')]
    
    items.update(dict(zip(keys, values)))
    
    items['адрес'] = soup.find(class_='table_map_link').text.replace('\xa0', ' ')
    items['цена'] = int(soup.find('div', {'class': 'realty_detail_price inline'}).text.replace('\xa0','').replace('руб.',''))
    items['ид'] = int(soup.find('strong').text)
    items['дата добавления'] = soup.find(class_='realty_detail_date nobr').get('title')
    items['дата истечения'] = soup.find_all(class_='realty_detail_date')[4].get('title')
    
    return items

In [7]:
def get_urls_pages(start_page=1, end_page=None):
    
    url_base = 'https://www.tomsk.ru09.ru/realty?type=1&otype=1&district[1]=on&district[2]=on&district[3]=on&district[4]=on&perpage=50&page='
    
    end_page = end_page or get_number_last_page()
    pages_to_parse = range(start_page, end_page + 1)
    urls_pages = [url_base + str(i) for i in pages_to_parse]
        
    return urls_pages

In [8]:
def get_urls_apartments_by_page(url_page):
    
    url_base = 'https://www.tomsk.ru09.ru'
    
    soup = get_soup_by_url(url_page)
    soup = soup.find_all('a', {'class':'visited_ads'})

    urls_apartments = set()
    
    for i in soup:
        urls_apartments.add(url_base + i.get('href'))
    
    return urls_apartments

In [9]:
def main(start_page=1, end_page=None, filename='data.json'):
    
    path = 'C:/Users/qwerty.Oleg/'
    
    if filename in listdir(path):
        with open(filename, 'r') as file:
            storage_dict = json.load(file)
    else:
        storage_dict = {}
    
    len_storage = len(storage_dict)
    print('Apartments in storage:', len_storage, '\n')
    
    urls_pages = get_urls_pages(start_page, end_page)
    
    for url_page in tqdm(urls_pages, desc='Pages'):
        urls_apartments = get_urls_apartments_by_page(url_page)
        urls_apartments_to_parse = urls_apartments.difference(set(storage_dict))
        
        if len(urls_apartments_to_parse) != 0:
            for url_apartment in tqdm(urls_apartments_to_parse, desc='Apartments', leave=False):
                storage_dict[url_apartment] = parse_apartment(url_apartment)
        
    with open(filename, 'w') as file:
        json.dump(storage_dict, file, ensure_ascii=False)
            
    print('New apartments:', len(storage_dict) - len_storage)

In [10]:
main()

Apartments in storage: 13392 




New apartments: 208


In [11]:
with open('data.json', 'r') as file:
    dict_data = json.load(file)
pd.DataFrame.from_dict(dict_data, orient='index')

,район,тип квартиры,количество комнат,общая площадь,этаж/этажность,отделка,санузел,этажность,материал,год постройки,вид,адрес,цена,ид,дата добавления,дата истечения,балкон/лоджия,жилая,кухня
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4353796,советский район,гостинка,1,18 кв.м,3/5,в хорошем состоянии,совмещенный,5,кирпич,1975,вторичное,Фрунзе проспект 126,1250000,4353796.0,15.03.2020 10:29:07,12.04.2020,NaN,NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4207812,советский район,NaN,4,200 кв.м,2/5,в отличном состоянии,раздельный,5,кирпич,1996,вторичное,Вершинина 7,13900000,4207812.0,12.03.2020 22:44:17,10.06.2020,"лоджия, остекление",NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4398582,советский район,NaN,3,81.4 кв.м,5/7,в отличном состоянии,раздельный,7,монолит,2009,вторичное,Источная 10,6900000,4398582.0,15.03.2020 10:29:09,12.04.2020,лоджия,NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4119652,советский район,секционка,2,33 кв.м,5/9,в отличном состоянии,раздельный,9,кирпич,1987,вторичное,Енисейская 4,1750000,4119652.0,15.03.2020 10:25:13,27.05.2020,NaN,NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4368552,ленинский район,гостинка,1,17.2 кв.м,3/5,NaN,NaN,5,кирпич,1972,вторичное,Заозерный переулок 16/2,1100000,4368552.0,15.03.2020 10:07:04,29.04.2020,NaN,13 кв.м,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4431120,октябрьский район,NaN,1,37.1 кв.м,5/14,в хорошем состоянии,совмещенный,14,кирпич,2012,вторичное,Иркутский тракт 183/1,2300000,4431120.0,28.04.2020 08:21:13,11.06.2020,"лоджия, остекление",NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4431144,кировский район,NaN,1,35.8 кв.м,7/12,в хорошем состоянии,совмещенный,12,кирпич,2010,вторичное,Богдана Хмельницкого переулок 8,1800000,4431144.0,28.04.2020 08:21:35,11.06.2020,"лоджия, остекление",NaN,11.6 кв.м
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4431098,советский район,NaN,2,48.22 кв.м,3/17,в отличном состоянии,совмещенный,17,монолит,NaN,новостройка,Маяковского 20,2950000,4431098.0,27.04.2020 21:22:27,26.07.2020,"лоджия, остекление",NaN,NaN
https://www.tomsk.ru09.ru/realty?subaction=detail&id=4431070,кировский район,NaN,1,34.8 кв.м,5/10,в отличном состоянии,совмещенный,10,кирпич,2020,вторичное,Нефтяная 15,2500000,4431070.0,27.04.2020 20:25:50,12.06.2020,"лоджия, остекление",NaN,NaN
